In [283]:
import numpy as np
import pandas as pd
from typing import Dict, Set, List
import copy

In [284]:
def populate_dictionaries_with_dataframe(
    df: pd.DataFrame,
    diseases_dict: Dict[str, Set[str]],
    symptoms_dict: Dict[str, Set[str]],
):
    lenght = len(df)
    attr_len = len(df.columns)

    for i in range(lenght):
        disease = (str)(df.iloc[i, 0])
        if disease in [
            "Allergy",
            "Varicose veins",
            "Paralysis (brain hemorrhage)",
            "Acne",
            "Heart attack",
            "Peptic ulcer diseae",
            "Drug Reaction",
            "Cervical spondylosis",
            "Hepatitis E",
            "hepatitis A",
            "Hepatitis B",
            "Hepatitis C",
            "Hepatitis D",
            "Fungal infection",
            "Hyperthyroidism",
            "Hypothyroidism",
            "Hypertension",
            "Hypoglycemia",
            "Alcoholic hepatitis",
            "Urinary tract infection",  # special case, this has 4 symptoms but each symptom only has this, hence when we are discarding symptoms with one disease, effectively no symptom is left for urinary track infection
        ]:
            continue
        if disease not in diseases_dict:
            diseases_dict.update({disease: set()})
        for j in range(1, attr_len):
            if not pd.isna(df.iloc[i, j]):
                symptom = (str)(df.iloc[i, j])
                if symptom in [
                    "mild_fever",
                    "family_history",
                    "irregular_sugar_level",
                    "watering_from_eyes",
                    "mood_swings",
                    "swelled_lymph_nodes",
                ]:
                    continue

                if symptom not in symptoms_dict:
                    symptoms_dict.update({symptom: set()})

                disease_symptoms_set = diseases_dict.get(disease)
                if disease_symptoms_set is not None:
                    disease_symptoms_set.add(symptom)

                symptom_diseases_set = symptoms_dict.get(symptom)
                if symptom_diseases_set is not None:
                    symptom_diseases_set.add(disease)

            else:
                break

In [285]:
def remove_single_disease_symptoms(
    disease_dict: Dict[str, Set[str]], symptoms_dict: Dict[str, Set[str]]
):
    ## removed symptoms list
    removed_symptoms: List[str] = []

    ## removing all symptoms which corresponds to only one disease
    for symptom in list(symptoms_dict):
        if len(symptoms_dict[symptom]) == 1:
            symptoms_dict.pop(symptom)
            removed_symptoms.append(symptom)

    for disease in list(disease_dict.keys()):
        for removed_symptom in removed_symptoms:
            if removed_symptom in disease_dict[disease]:
                disease_dict[disease].remove(removed_symptom)

        if len(disease_dict[disease]) == 0:
            disease_dict.pop(disease)

    return removed_symptoms

In [286]:
def generate_initial_questions(
    adj: List[List[str]],
    diseases: Set[str],
    ordered_symptoms: List[str],
):
    max_questions: int = 10
    max_diseases: int = 0
    # questions: List[str] = [""] * max_questions
    questions = [
        "high_fever",
        "vomiting",
        "fatigue",
        "headache",
        "chills",
        "cough",
        "skin_rash",
        "constipation",
        "swelling_joints",
    ]

    def dfs(
        adj: List[List[str]],
        idx: int,
        num_questions: int,
        curr_diseases: Set[str],
        symptoms: List[str],
        diseases: Set[str],
        ordered_symptoms: List[str],
    ):
        nonlocal max_questions, max_diseases, questions

        if curr_diseases == diseases:
            # shows how many diseases are covered
            max_diseases = max(max_diseases, len(curr_diseases))
            if max_questions > len(symptoms):
                print(
                    f"base case reached, updating question with number of questions {len(symptoms)}"
                )
                questions = copy.deepcopy(symptoms)
                max_questions = len(symptoms)
                return True
            # we can store the path and return if required the questions
            return False

        if num_questions >= max_questions:
            # if len(questions) > len(symptoms) and len(curr_diseases) > max_diseases:
            #     print(
            #         f"length of curr_diseases is {len(curr_diseases)} with depth {depth} and max_depth {max_depth}"
            #     )
            max_diseases = max(max_diseases, len(curr_diseases))
            #     questions = symptoms.copy()
            return False

        if idx >= len(adj):
            return False

        vis: list[bool] = [False] * len(adj[idx])
        for i, symptom in enumerate(adj[idx]):
            if symptom not in curr_diseases:
                vis[i] = True
            curr_diseases.add(symptom)

        # call once with taking that symptom
        symptoms.append(ordered_symptoms[idx])

        answer_found = dfs(
            adj,
            idx + 1,
            num_questions + 1,
            curr_diseases,
            symptoms,
            diseases,
            ordered_symptoms,
        )

        symptoms.remove(ordered_symptoms[idx])

        for i, symptom in enumerate(iterable=adj[idx]):
            if vis[i]:
                curr_diseases.remove(symptom)

        if answer_found:
            return True

        # call once without taking that symptom
        # now the tree is actually deeper than maximum number of questions already found
        # because this call where we do not select a question, this can do as deep as all the questions
        answer_found = dfs(
            adj,
            idx + 1,
            num_questions,
            curr_diseases,
            symptoms,
            diseases,
            ordered_symptoms,
        )

        if answer_found:
            return True
        else:
            return False

    # we have ran dfs once and hardcoded the questions for saving time
    # in case of change in diseases or symptoms, please run once agains
    # dfs(adj, 0, 0, set(), list(), diseases, ordered_symptoms)

    # print(f"Questions are :: {questions}")
    # print(f"Number of diseases covered by this questions are :: {max_diseases}")
    # print(f"Number of questions are :: {len(questions)}")
    return questions

In [287]:
def create_mapping(
    disease: str,  # the name of the disease in string format
    symptoms_mapping: Dict[str, int],
    disease_dict: Dict[str, Set[str]],
):
    """
    Given the name of the disease, this function will return the integer value i.e. bit mask
    corresponding to that disease based on the symptoms listen in disease dict corresponding to that disease.
    """
    mapping: int = 0
    disease_symptoms = disease_dict[disease]
    for symptom in disease_symptoms:
        mapping = mapping | (1 << symptoms_mapping[symptom])
    return mapping

In [288]:
def generate_diseases_map_from_list(
    diseases: List[str], diseases_mapping: Dict[str, int]
):
    combined_mask = 0
    for disease in diseases:
        combined_mask |= diseases_mapping[disease]
    return combined_mask

In [289]:
def generate_symptoms_map_from_list(
    symptoms: List[str], symptoms_mapping: Dict[str, int]
):
    combined_mask = 0
    for symptom in symptoms:
        combined_mask |= 1 << symptoms_mapping[symptom]
    return combined_mask

In [290]:
def generate_matching_diseases_from_symptoms_list(
    symptoms_list: List[str],
    symptoms_mapping: Dict[str, int],
    diseases_mapping: Dict[str, int],
):
    symptoms_mask = generate_symptoms_map_from_list(symptoms_list, symptoms_mapping)

    matching_diseases = [
        disease
        for disease in diseases_mapping
        if (diseases_mapping[disease] & symptoms_mask) == symptoms_mask
    ]

    return (symptoms_mask, matching_diseases)

In [291]:
def generate_matching_diseases_from_symptoms_mask(
    symptoms_mask: int,
    diseases_mapping: Dict[str, int],
):
    matching_diseases = [
        disease
        for disease in diseases_mapping
        if (diseases_mapping[disease] & symptoms_mask) == symptoms_mask
    ]

    return matching_diseases

In [292]:
def generate_next_set_questions(
    selected_symptoms: List[str],
    already_asked_mask: int,
    symptoms_mapping: Dict[str, int],
    diseases_mapping: Dict[str, int],
):
    _, matching_diseases = generate_matching_diseases_from_symptoms_list(
        selected_symptoms, symptoms_mapping, diseases_mapping
    )

    next_set_mask = (
        generate_diseases_map_from_list(matching_diseases, diseases_mapping)
        & ~already_asked_mask
    )

    next_set_questions: List[str] = []

    for symptom in symptoms_mapping:
        if 1 << symptoms_mapping[symptom] & next_set_mask != 0:
            next_set_questions.append(symptom)

    return (next_set_mask, next_set_questions)

In [293]:
df = pd.read_csv(filepath_or_buffer="./archive/dataset.csv", skipinitialspace=True)

In [294]:
""" stores the diseaseas key and a set containing symptoms for that disease as the value """
disease_dict: Dict[str, Set[str]] = {}
symptoms_dict: Dict[str, Set[str]] = {}

populate_dictionaries_with_dataframe(df, disease_dict, symptoms_dict)
remove_single_disease_symptoms(disease_dict, symptoms_dict)
diseases: Set[str] = set(disease_dict.keys())  # contains all the diseasein a set format

In [295]:
# stores the symtoms and the number of diseases that is possible if that symptom is present
symptom_diseases_count = [
    (symptom, len(symptoms_dict[symptom])) for symptom in symptoms_dict
]
symptom_diseases_count.sort(key=lambda x: x[1], reverse=True)

In [296]:
"""adj[i] corresponds to the symptoms at ith position of ordered symptoms"""
ordered_symptoms = [x[0] for x in symptom_diseases_count]
adj: List[List[str]] = [list(symptoms_dict[symptom]) for symptom in ordered_symptoms]

In [297]:
# this does not store the actually bit shifted number, rather the amount 1 needs to be shifted
symptoms_mapping: Dict[str, int] = {
    symptom: idx for idx, symptom in enumerate(reversed(ordered_symptoms))
}

diseases_mapping: Dict[str, int] = {
    disease: create_mapping(disease, symptoms_mapping, disease_dict)
    for disease in disease_dict
}

In [298]:
initial_questions = generate_initial_questions(adj, diseases, ordered_symptoms)
initial_mask = 0
for question in initial_questions:
    initial_mask |= 1 << symptoms_mapping[question]

already_asked_mask = initial_mask
selected_symptoms_mask = 0


selected_symptoms = ["high_fever"]
symptoms_mask = generate_symptoms_map_from_list(selected_symptoms, symptoms_mapping)
next_set_mask, next_set_questions = generate_next_set_questions(
    selected_symptoms, already_asked_mask, symptoms_mapping, diseases_mapping
)
already_asked_mask |= next_set_mask
selected_symptoms_mask |= symptoms_mask

In [303]:
generate_matching_diseases_from_symptoms_list(
    selected_symptoms, symptoms_mapping, diseases_mapping
)

(2147483648,
 ['AIDS',
  'Bronchial Asthma',
  'Jaundice',
  'Malaria',
  'Chicken pox',
  'Dengue',
  'Typhoid',
  'Tuberculosis',
  'Common Cold',
  'Pneumonia',
  'Impetigo'])

In [302]:
initial_questions

['high_fever',
 'vomiting',
 'fatigue',
 'headache',
 'chills',
 'cough',
 'skin_rash',
 'constipation',
 'swelling_joints']

In [300]:
print(next_set_questions)

['red_spots_over_body', 'lethargy', 'yellowing_of_eyes', 'yellowish_skin', 'phlegm', 'joint_pain', 'muscle_pain', 'sweating', 'breathlessness', 'diarrhoea', 'weight_loss', 'abdominal_pain', 'itching', 'loss_of_appetite', 'malaise', 'nausea', 'chest_pain']


In [299]:
generate_matching_diseases_from_symptoms_mask(selected_symptoms_mask, diseases_mapping)

['AIDS',
 'Bronchial Asthma',
 'Jaundice',
 'Malaria',
 'Chicken pox',
 'Dengue',
 'Typhoid',
 'Tuberculosis',
 'Common Cold',
 'Pneumonia',
 'Impetigo']